# UI

> A chat interface UI demo

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.all import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from fastcore.utils import *
import nbformat
import itertools
from functools import wraps

### Initial setup

In [ ]:
custom_highlighting = " ".join(name for name in globals() if name[0].isupper())
ftrs = CodeHighlightThemeScript(custom_highlighting)

In [ ]:
app = FastHTML(ftrs=ftrs, exts=("ws", "preload"), session_cookie="mysession")
rt = app.route

In [ ]:
@app.middleware("http")
async def add_cache_headers(request, call_next):
    response = await call_next(request)
    if "/documentation-rag/" not in str(request.url):
        response.headers["Cache-Control"] = "max-age=300"  # 5 minutes
    return response

In [ ]:
srv = JupyUvi(app)

In [ ]:
slug_map = {}

In [ ]:
def make_key():
    current = [None]
    def key_fn(cell):
        if cell.cell_type == "markdown" and cell.source.startswith("###"):
            current[0] = cell.source.strip("###").strip()
        return current[0]
    return key_fn

In [ ]:
def mk_nbdict(filename:str):
    nb = nbformat.read(filename, as_version=4)
    cells = itertools.islice(itertools.dropwhile(lambda c: c.source != "# Components", nb.cells), 1, None)
    nbdict = {}
    for header, group in itertools.groupby(cells, key=make_key()):
        header_slug = slugify(header)
        slug_map[header_slug] = header
        nbdict[header_slug] = list(group)
    return nbdict 


In [ ]:
mk_nbdict("03_chat.ipynb").keys()

dict_keys(['chat-input', 'chat-message', 'chat-code', 'chat-pairs'])

In [ ]:
matched = re.search(r"\d+_(\w+)\.ipynb", "01_common.ipynb").group(1)
matched

'common'

In [ ]:
def format_nm(nm:str):
    return re.search(r"\d+_(\w+)\.ipynb", nm).group(1).title()

In [ ]:
documentation = {format_nm(nm): mk_nbdict(nm) for nm in ["01_common.ipynb", "02_interactive.ipynb", "03_chat.ipynb"]}

In [ ]:
documentation.keys()

dict_keys(['Common', 'Interactive', 'Chat'])

### Search bar

In [ ]:
def SearchBarButton(id):
    return DialogOpenButton(
        Div(
            Div(
                Icon("search", cls="size-4"),
                Span("Search Documentation", cls="text-sm"),
                cls="flex gap-2 items-center",
            ),
            Kbd("⌘ K", cls="text-2xs"),
            cls="flex items-center px-3 py-2 justify-between",
        ),
        cls="m-1 hover:cursor-pointer bg-background rounded-lg w-full",
        did=id,
    )

In [ ]:
def mk_command_items(category:str, nms:list[str]):
    return [
        A(
            CommandItem(slug_map[nm]), 
            href=f"/{category.lower()}/{nm}",
            cls="block w-full",
            onclick="this.closest('dialog').close(); document.activeElement.blur()"
        )
        for nm in nms
    ]

In [ ]:
def SearchBarCommand(id):
    groups = [
        CommandGroup(
            ItemHeader(category), 
            *mk_command_items(category, nbdict.keys())
        )
        for category, nbdict in documentation.items()
    ]
    
    return CommandDialog(
        Command(
            CommandSearch(),
            CommandScrollable(*groups, style="height: 400px;"),
            cls="w-4xl",
            style="top: 29%; max-width: 650px; max-height: 500px;",

        ),
        id=id,


    )

In [ ]:
def SearchBar(id="search-bar"):
    return Div(
        SearchBarButton(id=id),
        SearchBarCommand(id=id),
        Script(f"""
        document.addEventListener('keydown', (e) => {{
            if ((e.ctrlKey || e.metaKey) && e.key === 'k') {{
                e.preventDefault();
                const dialog = document.getElementById("{id}");
                if (dialog.open) {{
                    dialog.close();
                }} else {{
                    dialog.showModal();
                    dialog.querySelector("header input")?.focus();
                }}
            }}
        }});
        """),
        cls="flex w-full",
    )

### Sidebar

In [ ]:
def DropdownFooter():
    return Div(
            Dropdown(
                DropdownItem("Upgrade to Pro"),
                Separator(),
                DropdownItem("Account"),
                trigger_btn=DropdownTriggerButton(
                    Div(
                        Avatar(src="https://ui.shadcn.com/avatars/shadcn.jpg"), Span("Taya", cls="pl-2"),
                        cls="flex items-center w-full",
                    ),
                    Icon("chevrons-up-down"), did="sidebar-dropdown", 
                    cls=f"{ButtonT.ghost} justify-between py-6 px-3 m-1",
                    style="width: calc(100% - 0.5rem)"
                ),
                id="sidebar-dropdown",
                side="right", align="end",
        ),
        cls="flex flex-col items-left w-full",
    )

In [ ]:
category_icons = {"Common": "blocks", "Interactive": "mouse-pointer-click", "Chat": "message-square"}

In [ ]:
def SiteSidebar():
    contents = [
        SidebarCollapsable(
            IconTitle(category, icon=category_icons.get(category)), 
            [slug_map[k] for k in nbdict.keys()],
            href_list=nbdict.keys(),
            href_parent=category,
        ) 
        for category, nbdict in documentation.items()
    ]
    return Sidebar(
        SearchBar(),
        Group(
            "Components", 
            *contents
        ),
        SidebarGroup("Demos", ["Documentation RAG"], icon_list=["bot"], href_parent="demos"),
        footer=DropdownFooter(),
        hx_boost="true", hx_target="#main-content", hx_select="#main-content", hx_ext="preload", preload="mouseover",
    )

In [ ]:
def TopBar(category="", component=""):
    return Div(
        Div(
            ToggleButton(),
            SeparatorVertical(),
            Breadcrumb([category.title(), slug_map.get(component, component.title())]),
            cls="flex items-center gap-2"
        ),
        ThemeSwitcher(),
        cls="flex justify-between items-center w-full px-3 overflow-hidden py-1 border-b"
    )

In [ ]:
def sidebar_layout(fn):
    @wraps(fn)
    def inner_fn(*args, **kwargs):
        return Div(
            SiteSidebar(),
            Main(
                TopBar(**kwargs),
                Div(
                    fn(*args, **kwargs),
                    cls="flex flex-col flex-1 overflow-y-auto min-h-0",
                ),
                cls="h-full flex flex-col",
                id="main-content"
            ),
            cls="h-screen w-full",
        )
    return inner_fn

### Chat interface

In [ ]:
from lisette import *
chat = Chat(model="anthropic/claude-3-haiku-20240307")
chat

<lisette.core.Chat>

In [ ]:
# for r in chat("Testing2", stream=True):
#     delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
#     if delta and delta.content:
#         print(delta.content, end='', flush=True)
# final_response = r.choices[0].message.content

In [ ]:
@app.ws("/send_msg_ws")
async def send_msg_ws(text:str, send):
    await send(Div(ChatPrompt(text), id="chat-interface", hx_swap_oob="beforeend"))
    r = chat(text, stream=False)
    output = r.choices[0].message.content
    await send(Div(ChatAssistant(output), id="chat-interface", hx_swap_oob="beforeend"))

In [ ]:
def RAGChat():
    return ChatInterface(
            ChatPrompt("A test message sent by a user\n\nAnd new lines\nAre here."),
            ChatAssistant("A smaller response returned by a LLM. Great question. You are absolutely right!"),
            chat_input=ChatInput(
                cls="mb-1 mt-0", ta_cls="rounded-t-none", width="w-full",
                hx_ext="ws", ws_connect="/send_msg_ws", ws_send=True,
            ),
            cls="flex-1",
            id="chat-interface",
        )

In [ ]:
@rt("/demos/documentation-rag")
@sidebar_layout
def documentation_rag():
    return Div(
        RAGChat(),
        cls="flex flex-1",
    )

### Dyanmic routes

#### Display demo card

In [ ]:
def find_demos(nb_cell_list:list):
    return L(nb_cell_list).filter(Self.source.startswith("#| demo"))

In [ ]:
def exec_demo(cell:dict):
    ns = {}
    exec(cell.source, globals(), ns)
    name, fn = next(iter(ns.items())) 
    return (name, fn, cell.source)

In [ ]:
def render_demo_cards(nb_cell_list):
    return [DisplayDemoCard(*exec_demo(d)) for d in find_demos(nb_cell_list)]

In [ ]:
def DisplayDemoCard(nm, fn, src):
    return Div(
        Tabs(
            contents=[
                CardOutline(fn(), cls="rounded-lg min-h-96 flex justify-center items-center py-5"),
                Card(
                    Pre(
                        Code(src, cls="language-python-custom", style="background: transparent; overflow: visible;"),
                    ), 
                cls="rounded-lg min-h-96 flex text-sm overflow-auto overflow-y-auto",
                ),
            ],
            tablist=["preview", "code"],
            id=f"{nm}-tab",
            cls="w-full",
            nav_cls="w-fit ml-auto",
        ),
        cls="w-full",
    )

#### Source code accordion

In [ ]:
def strip_meta(text, prefixes=["#| exports", "#| export", "\n"]):
    for prefix in prefixes:
        text = text.removeprefix(prefix)
    return text

In [ ]:
def find_sourcecode(nb_cell_list):
    return L(nb_cell_list).filter(Self.source.startswith(("#| demo", "#| preview")), negate=True)[1:]

In [ ]:
def get_code_and_output(cell):
    code = strip_meta(cell["source"])
    output = [strip_meta(get_output_text(o)) for o in cell["outputs"]]
    return code, output

In [ ]:
def get_clean_output(output):
    if not isinstance(output, dict): return str(output)
    data = output.get("data", output)
    text = data.get("text/plain") or data.get("text/html") or data.get("text", "")
    return strip_meta(text)

In [ ]:
def render_cell(c):
    if c["cell_type"] == "markdown": return ChatNote(c["source"])
    code = strip_meta(c["source"])
    out = get_clean_output(c["outputs"][0]) if c["outputs"] else None
    return ChatCodeAndOutput(code, out)

In [ ]:
def AccordionSourceCode(nb_cell_list):
    contents = find_sourcecode(nb_cell_list).map(render_cell)
    return Accordion(
        AccordionItem(
            "Source code",
            ChatInterface(
                *contents,
                inner_cls="border rounded-lg",
            )
        ),
        cls="mt-4"
    )

#### Default component route

In [ ]:
@rt("/{category}/{component}")
@sidebar_layout
def get(category: str, component: str):
    nb_cell_list = documentation.get(category.title(), {}).get(component, {})
    if not nb_cell_list:
        return NotFoundError("Page not found...")
    return Div(
        H3(slug_map[component], cls="text-xl font-semibold tracking-tight mt-3"),
        Div(*render_demo_cards(nb_cell_list), cls="flex flex-col gap-3"),
        AccordionSourceCode(nb_cell_list),
        cls="flex flex-col w-full lg:max-w-4xl mx-auto",
    )

In [ ]:
# This route needs to be defined in the demo for the toast example to work
@rt
def toast_example():
    return Toast("Success", "Toast send from backend!")